# Match voertuig in ongeval met voertuig in RDW database

Met dit script kan het kenteken van een voertuig in een verkeersongeval worden achterhaald, waar deze kentekens eerder uit de ongevallen database zijn verwijderd, met het doel deze te anonimiseren. Wanneer het kenteken van een voertuig niet kan worden vastgesteld, wordt het aantal overgebleven resultaten genoteerd.

Het matchen van voertuigen gebeurt in de onderste cell van dit notebook, ik heb hier een stukje "machine learning" voor bedacht, om dit eens te proberen, niet omdat het handig is in deze situatie. 

Voor elk voertuig in de verkeersongevallen database wordt deze vergeleken met de RDW database op acht parameters.
Omdat de RDW database te groot is voor mijn RAM geheugen heb ik deze in delen ingeladen (alleen voertuigen met hoofdkleur x). Zo zijn dus eerste alle zwarte voertuigen onderzocht, gevolgt door wit, etc.

De volgorde waarin de overige parameters worden gebruikt is voor het eerste voertuig willekeurig en gelijk verdeeld. Deze veranderd naarmate een vergelijking met de specifieke parameters succesvol(0 < N_new < N_old) is geweest, doormiddel van een counter. Een hogere counter waarde zorgt dat de parameter meer kans heeft om gekozen te worden voor de vergelijking. Na 100.000 vergelijkingen wordt een vaste volgorde vast gesteld, waarin de variabelen met de hoogste counter waarde eerst wordt genomen afnemend naar de minste waarde.


In [2]:
from datetime import date
import math
import numpy as np
from pyspark.sql import Row
import pandas as pd
from random import randint
from pyspark.sql import SQLContext

#PRE-INSTELLEN DATABASE RDW
rdwdf = spark.read.csv(
    "C:/Data/Open_Data_RDW__Gekentekende_voertuigen.csv",
    header=True,
    mode="FAILFAST"
)
rdwdf = rdwdf.withColumnRenamed("Eerste kleur", "Hoofdkleur").withColumnRenamed("Tweede kleur", "Subkleur").withColumnRenamed("Datum eerste afgifte Nederland", "Eerste_inschrijving").withColumnRenamed("Massa ledig voertuig", "Massa_leeg").withColumnRenamed("Datum eerste toelating", "Eerste_toelating").withColumnRenamed("Voertuigsoort", "Voertuig_id")

rdwdf = rdwdf.withColumn("Eerste_inschrijving", rdwdf["Eerste_inschrijving"].cast("string"))
rdwdf = rdwdf.withColumn("Massa_leeg", rdwdf["Massa_leeg"].cast("string"))
rdwdf = rdwdf.withColumn("Eerste_toelating", rdwdf["Eerste_toelating"].cast("string"))
rdwdf = rdwdf.withColumn("Voertuig_id", rdwdf["Voertuig_id"].cast("string"))

rdwdf.createOrReplaceTempView("rdw")

In [3]:
#DATABASE ONGEVALLEN

dataDir = r"D:\Traineeship\data"


def MakeDict(l1, l2):
    NameTypeDict={}
    for i in range(len(l1)):
        NameTypeDict.update({l1[i]:l2[i]})
    return NameTypeDict

def Round(x):  
    return str(round(x))

def StringToDate(x):
    x = str(x)
    year = x[0:4]
    month = x[4:6]
    day = x[6:8]
    return day+"/"+month+"/"+year

def Date2Date(x):
    x = str(x)
    year = x[0:4]
    month = x[5:7]
    day = x[8:10]
    return day+"/"+month+"/"+year



In [4]:
namenongevallen=["VKL_NUMMER", "REGNUMMER", "PVOPGEM", "DATUM_VKL", "DAG_CODE", "MND_NUMMER", "JAAR_VKL", "TIJDSTIP", "UUR", "DDL_ID", "AP3_CODE", "AP4_CODE", "AP5_CODE","ANTL_SLA", "ANTL_DOD", "ANTL_GZH", "ANTL_SEH","ANTL_GOV", "ANTL_PTJ", "ANTL_TDT", "MNE_CODE", "AOL_ID", "NIVEAUKOP","WSE_ID","WSE_AN","BEBKOM","MAXSNELHD", "WVL_ID", "WVG_ID","WVG_AN", "WDF_ID","WDK_AN", "LGD_ID", "ZAD_ID", "WGD_CODE_1", "WGD_CODE_2", "BZD_ID_VM1", "BZD_ID_VM2", "BZD_ID_VM3", "BZD_VM_AN", "BZD_ID_IF1", "BZD_ID_IF2", "BZD_ID_IF3", "BZD_IF_AN","BZD_ID_TA1","BZD_ID_TA2", "BZD_ID_TA3", "BZD_TA_AN", "JTE_ID", "WVK_ID", "HECTOMETER","FK_VELD5", "HUISNUMMER","GME_ID", "GME_NAAM", "PVE_CODE", "PVE_NAAM", "KDD_NAAM","PLT_NAAM", "DIENSTCODE", "DIENSTNAAM", "DISTRCODE", "DISTRNAAM"]
typesongevallen=[str, str, str, str, str, int, int, str, str, str, str, str, str, int, int ,int ,int ,int ,int , int, str, str, str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str]
naamongevallen=["Ongeluk_id", "Regnummer", "Procesverbaal", "Datum", "Dag", "Maand", "Jaar", "Tijdstip", "Uur", "Dagdeel", "Afloop3", "Afloop4", "Afloop5", "#Slachtoffers", "#Doden", "#Ziekenhuis", "#Spoedeisend", "#Overig", "#Partijen", "#Toedrachten", "Manouvre_id", "Aard_id", "Niveaukop", "Wegsituatie", "Wegsituatie2", "Bebouwde_kom", "Maxsnelheid", "Wegverlichting", "Wegverharding", "Wegverharding2", "Wegdek_id", "Wegdek", "Licht_id", "Zicht_id", "Weer", "Weer2", "Bijzonderheid", "Bijzonderheid2", "Bijzonderheid3", "Bijzonderheid4", "Bijzonderheid_id1", "Bijzonderheid_id2", "Bijzonderheid_id3", "Bijzonderheid_id4", "Bijzonderheid_tijdelijk1", "Bijzonderheid_tijdelijk2", "Bijzonderheid_tijdelijk3", "Bijzonderheid_tijdelijk4", "Junctie_id", "Wegvakt_id", "Hectometer", "Puntlocatie", "Huisnummer", "Gemeente_id", "Gemeente", "Provincie_id", "Provincie", "Kader_id", "District_id", "Dienstcode", "Dienstnaam", "District_id", "District"]
DICTongevallen=MakeDict(naamongevallen, typesongevallen)

namenpartijen = ['PTJ_ID', 'VKL_NUMMER', 'NUMMER', 'DOORRIJDER', 'OTE_ID', 'OTE_AN', 'NTT_CODE_V', 'VTGVERZ', 'SCHADE', 'GETRAANH', 'GEVSTOF', 'VTGVERL', 'ANTL_PAS', 'GEBDAT', 'GEBJAAR', 'LEEFTIJD', 'LKE_ID', 'NTT_CODE_B', 'GESLACHT', 'BLAASTEST', 'ART8', 'MEDICGEBR', 'RIJBEWGEL', 'RIJBEWCAT', 'RIJBEWBEG', 'BROMFCERT', 'UITGPOS1', 'UITGPOS2', 'UITGPOS_AN', 'VOORGBEW', 'AGT_TYPE', 'AGT_ID_1', 'AGT_ID_2', 'BWG_ID_1', 'BWG_ID_2', 'BWG_AN', 'TDT_ID_1', 'TDT_ID_2', 'TDT_ID_3', 'TDT_AN']
typespartijen = [str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str]
naampartijen =  ["Partij_id", "Ongeluk_id", "Volgnummer", "Doorrijder", "Object_id", "Object_id2", "Nationaliteit_voertuig", "Verzekerd", "Schade", "Aanhangwagen", "Gevaarlijke_stoffen", "Verlichting", "#Passagiers", "Geboortedatum", "Geboorte_jaar", "Leeftijd", "Leeftijd_id", "Nationaliteit_bestuurder", "Geslacht", "Blaastest", "Artikel8", "Medicijnen/drugs", "Bezit_rijbewijs", "Rijbewijs_categorie", "Beginner", "Bromfiets_certificaat", "Locatie_voor1", "Locatie_voor2", "Locatie_voor3", "Manouvre", "Aangrijppunt", "Aangrijppunt_id1","Aangrijppunt_id2", "Beweging1", "Beweging2", "Beweging3", "Toedracht_id1", "Toedracht_id2", "Toedracht_id3", "Toedracht_id4"]
DICTpartijen=MakeDict(naampartijen, typespartijen)

namenvoertuigen = ['PTJ_ID', 'PEILDAT', 'VCE_CODE',  'TVT_CODE_1',  'TVT_CODE_2',  'TVE_CODE',  'TGKNR_VTG',  'TGKNR_23W',  'UITV_VLGNR',  'UITV_WYZNR',  'NAAM_FABR',  'RBD_CODE_1',  'VARDL_1_1',  'VARDL_1_2',  'RBD_CODE_2',  'VARDL_2_1',  'VARDL_2_2',  'IRG_CODE',  'MERKBESCHR',  'MERKCODE',  'TYPBESCHR',  'EERINSDAT',  'EERTOEDAT',  'BSF_CODE_H',  'BSF_CODE_N',  'MASSALEEG',  'MASSABDKL',  'MAXMASSA',  'ANTL_AS',  'ANTL_CIL',  'ANTL_DEUR',  'ANTL_WIEL',  'BREEDTE',  'LENGTE',  'AFSTKOP_AZ',  'AFSTKOP_VZ',  'ASNUMMER_1',  'WGG_CODE_1',  'SPRBRDTE_1',  'ASNUMMER_2',  'WGG_CODE_2',  'SPRBRDTE_2',  'WIELBASIS',  'KLR_CODE_H',  'KLR_CODE_N',  'CILINHOUD',  'VERMOGEN',  'GELNIV',  'TRNGELNIV',  'MELDDATAPK',  'VERVDATAPK']
typesvoertuigen = [str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str,str,str,float,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str]
naamvoertuigen =  ["Partij_id", "Peildatum", "Voertuig_id", "Toevoeging1", "Toevoeging2", "Toevoeging3", "Goedkeuringsnummer", "Goedkeuringsnummer2", "Uitvoeringsvolgnummer", "VUitvoeringsvolgnummer2", "Fabrikant", "Bijzonderheid1", "Bijzonderheid2", "Bijzonderheid3", "Bijzonderheid4", "Bijzonderheid5", "Bijzonderheid6", "Inrichting", "Merk", "Merk_id", "Handelsbenaming", "Eerste_inschrijving", "Eerste_toelating", "Brandstof1", "Brandstof2", "Massa_leeg", "Massa_bedrijfsklaar", 'Max_massa', "#Assen", "#Cilinders", "#Deuren", "#Wielen", "Breedte", "Lengte", "Afstand_kop_achter", "Afstand_kop_voor", "As_nummer1", "Weggedrag1", "Spoorbreedte1", "As_nummer2", "Weggedrag2", "Spoorbreedte2", "Wielbasis", "Hoofdkleur", "Subkleur", "CC", "PK", "Geluid", "Toerental_geluid", "Meld_APK", "Verval_APK"]
DICTvoertuigen=MakeDict(namenvoertuigen, typesvoertuigen)

DICTaard = {"3":"Dier", "9":"Eenzijdig", "7":"Flank", "6":"Frontaal", "2":"Geparkeerd voertuig", "8":"Kop/staart", "5":"Los voorwerp", "0":"Onbekend", "4":"Vast voorwerp", "1":"Voetganger"}

DICTvoertuigenid = {1:"personenauto", 3:"bedrijfsauto", 5:"bedrijfsauto met koppeling", 6:"aanhangwagen", 7:"oplegger", 8:"autonome aanhangwagen", 9:"middenasaanhangwagen", 12:"motorfiets", 13:"motorfiets met zijspan", 14:"driewielige motorcarrier", 15:"lichte invalidenwagen", 18:"driewielig motorvoertuig.", 19:"bromfiets"}

DICTkleuren = {
0: "ORANJE",
1: "ROSE",
2: "ROOD",
3: "WIT",
4: "BLAUW",
5: "GROEN",
6: "GEEL",
7: "GRIJS",
8: "BRUIN",
9: "BEIGE",
10: "CREME",
11: "PAARS",
12: "ZWART",
13: "DIVERSEN"
}

In [5]:
ongevallen2014df = pd.read_csv(dataDir + r"\2014\Ongevallengegevens\ongevallenUTF.txt", delimiter=",", skipinitialspace=True,skiprows=1, skip_blank_lines=True, names=naamongevallen, comment="#", parse_dates=['Datum'], dtype=DICTongevallen, index_col=False)
ongevallen2015df = pd.read_csv(dataDir + r"\2015\Ongevallengegevens\ongevallenUTF.txt", delimiter=",", skipinitialspace=True,skiprows=1, skip_blank_lines=True, names=naamongevallen, comment="#", parse_dates=['Datum'], dtype=DICTongevallen, index_col=False)
ongevallen2016df = pd.read_csv(dataDir + r"\2016\Ongevallengegevens\ongevallenUTF.txt", delimiter=",", skipinitialspace=True,skiprows=1, skip_blank_lines=True, names=naamongevallen, comment="#", parse_dates=['Datum'], dtype=DICTongevallen, index_col=False)
ongevallendf = pd.concat([ongevallen2014df,ongevallen2015df,ongevallen2016df])
ongevallendf = ongevallendf.reset_index()
ongevallendf.replace({"Aard_id": DICTaard}, inplace=True)
ongevallendf = ongevallendf[pd.isnull(ongevallendf["Tijdstip"]) == False]

partijen2014df = pd.read_csv(dataDir + r"\2014\Ongevallengegevens\partijen.txt", delimiter=",", names = naampartijen, parse_dates=['Geboortedatum'], skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTpartijen, index_col=False)
partijen2015df = pd.read_csv(dataDir + r"\2015\Ongevallengegevens\partijen.txt", delimiter=",", names = naampartijen, parse_dates=['Geboortedatum'], skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTpartijen, index_col=False)
partijen2016df = pd.read_csv(dataDir + r"\2016\Ongevallengegevens\partijen.txt", delimiter=",", names = naampartijen, parse_dates=['Geboortedatum'], skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTpartijen, index_col=False)
partijen2014df["Jaar"] = 2014
partijen2015df["Jaar"] = 2015
partijen2016df["Jaar"] = 2016
partijendf = pd.concat([partijen2014df,partijen2015df,partijen2016df])
partijendf = partijendf.reset_index()

voertuigen2014df = pd.read_csv(dataDir + r"\2014\Voertuigkenmerkgegevens\voertuigdetails.txt", delimiter=",", names = naamvoertuigen, parse_dates=["Peildatum", "Eerste_inschrijving", "Eerste_toelating", "Meld_APK", "Verval_APK"],  skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTvoertuigen, index_col=False)
voertuigen2015df = pd.read_csv(dataDir + r"\2015\Voertuigkenmerkgegevens\voertuigdetails.txt", delimiter=",", names = naamvoertuigen, parse_dates=["Peildatum", "Eerste_inschrijving", "Eerste_toelating", "Meld_APK", "Verval_APK"],  skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTvoertuigen, index_col=False)
voertuigen2016df = pd.read_csv(dataDir + r"\2016\Voertuigkenmerkgegevens\voertuigdetails.txt", delimiter=",", names = naamvoertuigen, parse_dates=["Peildatum", "Eerste_inschrijving", "Eerste_toelating", "Meld_APK", "Verval_APK"],  skipinitialspace=True,skiprows=1, skip_blank_lines=True, comment="#", dtype=DICTvoertuigen, index_col=False)
voertuigen2014df["Jaar"] = 2014
voertuigen2015df["Jaar"] = 2015
voertuigen2016df["Jaar"] = 2016
voertuigendf = pd.concat([voertuigen2014df,voertuigen2015df,voertuigen2016df])
voertuigendf = voertuigendf.reset_index()
voertuigendf["Voertuig_id"].astype(int)
voertuigendf.replace({"Voertuig_id": DICTvoertuigenid}, inplace=True)
voertuigendf.replace({"Hoofdkleur": DICTkleuren}, inplace=True)
voertuigendf.replace({"Subkleur": DICTkleuren}, inplace=True)
voertuigendf["Hoofdkleur"].astype(str)
voertuigendf["Subkleur"].astype(str)

del ongevallendf["index"]
del partijendf["index"]
del voertuigendf["index"]

ongevallendf.sort_index(inplace=True)
partijendf.sort_index(inplace=True)
voertuigendf.sort_index(inplace=True)

c:\users\evert\appdata\local\programs\python\python36\lib\site-packages\pandas\io\parsers.py:709: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
c:\users\evert\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\evert\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
subdf=pd.merge(partijendf, ongevallendf, left_on=["Ongeluk_id", "Jaar"], right_on=["Ongeluk_id", "Jaar"], how='outer').sort_values("Ongeluk_id")
subdf.sort_values("Partij_id", inplace=True)
voertuigendf["Partij_id"]=voertuigendf["Partij_id"].apply(str)
df=pd.merge(subdf, voertuigendf, left_on=["Partij_id", "Jaar"], right_on=["Partij_id", "Jaar"], how='left').sort_values("Ongeluk_id")
df['Hoofdkleur']=df['Hoofdkleur'].apply(str)
df = df.drop(df[(df.Hoofdkleur == 'nan') | (df.Hoofdkleur == '20131022.0')].index)
df = df.sort_values(['Hoofdkleur'], ascending=False)
df.reset_index()
df = df.rename(columns={'Voertuig_id': 'Voertuigsoort'})
df["Massa_leeg"] = df["Massa_leeg"].fillna(value=0)
df["Massa_leeg"] = df["Massa_leeg"].apply(Round)
df["Eerste_toelating"] = df["Eerste_toelating"].apply(StringToDate) 
df["Eerste_inschrijving"] = df["Eerste_inschrijving"].apply(Date2Date) 

In [ ]:
#ZOEKEN EN VERGELIJKEN

df2 = df


#Import, dict, setdtypes

def DictNameVal(l1):
    nameTypeDict = {}
    for i in range(len(l1)):
        nameTypeDict.update({l1[i]: str})
    return nameTypeDict

def DictKnown(heads):
    knowDict = {}
    for i in range(1, len(heads)):
        knowDict.update({heads[i]: 1})
    return knowDict

def LoadDataBase1(kleur):
    firstString = "SELECT Kenteken, Voertuig_id, Merk, Eerste_toelating, Eerste_inschrijving, Hoofdkleur, Subkleur, Massa_leeg, Handelsbenaming FROM rdw "
    middleString = "WHERE Hoofdkleur == "
    kleurInvullen = "'"+kleur+"'"
    lastString = kleurInvullen
    sqlLijst = spark.sql(firstString+middleString+lastString)
    df1 = sqlLijst.toPandas()    
    return df1

# Set amount of rows in df1
df1RowsCount = rdwdf.count()
df2ColNamen = list(df2)
df2NamesValues = DictNameVal(df2ColNamen)
df2Known = DictKnown(df2ColNamen)
df2Learnt = df2Known.copy()
#Set index
index = 0
summ = 0
counterr = 0
listt = []
#Set amount of columns/rows in df2
df2RowsCount = int(df2.count().mean())
#Column extra list
addToList = []
#Set automerk
kleur = "ZWART"
df1 = LoadDataBase1(kleur)


#--------------------------LOOP-------------------------- Rows

while True:
    if kleur != df1["Hoofdkleur"].iloc[index]:
        df1 = LoadDataBase1(kleur)
    
    df2KnownTemp = df2Known.copy()
    #Rows in df1
    rowsLeftdf = df1
    rowsLeftCount = df1RowsCount
    #HIER MOGELIJK FOUTMELDING CHECK DIE KROMMEHAAKJES
    resultsVar = df2[["Ongeluk_id", "Partij_id"]].iloc[index]
    resultsdf = pd.DataFrame(columns=['Kenteken', "Zekerheid", "Ongeluk_id", "Partij_id"])

    #-------------------------LOOP--------------------------- Columns
    while True:
        # Select random column
        if summ < 100000:
            summ = 0
            for i in df2KnownTemp:
                summ += df2KnownTemp[i]
            randomColumn = randint(0, summ)
            summ = 0
            for i in df2KnownTemp:
                summ += df2KnownTemp[i]
                if summ >= randomColumn:
                    columnIndex = (i)
                    break
            df2ColName = columnIndex
            df2KnownTemp.pop(columnIndex)
        else:
            if counterr == 0:
                dict2 = sorted(dict.items(),reverse=True, key=operator.itemgetter(1))
                for i in df2Learnt:
                        for ii in i:
                            if counterr %2 == 0:
                                counterr +=1
                                listt.append(ii)
                            else:
                                counterr += 1
            df2ColName = listt[0] 
            listt.remove(listt[0])
            df2KnownTemp = listt

        #Row selecteren (column zit hier in)
        df2Var = (df2[str(df2ColName)].iloc[index])
        
        #Get rows left
        try:
            rowCount = int(rowsLeftdf.loc[rowsLeftdf[df2ColName] == df2Var].count().mean())
            if rowCount>0:
                rowsLeftCount = rowCount
                rowsLeftdf = rowsLeftdf.loc[rowsLeftdf[df2ColName] == df2Var]
                df2Learnt[df2ColName] += 1
        except KeyError:
            fout = 0
        except ValueError:
            print("ValueError")
        except TypeError:
            print("TypeError")

        if rowsLeftCount == 1:
            resultsVar["Kenteken"] = rowsLeftdf["Kenteken"].iloc[0]
            resultsVar["Zekerheid"] = 100
            resultsVar["Index"] = index
            resultsdf = resultsdf.append(resultsVar, ignore_index=True)
            with open("D:\Traineeship\Data\Output\Results.csv", 'a') as f:
                resultsdf.to_csv(f, header=False)
            index += 1
            break
        if len(df2KnownTemp) == 0:
            resultsVar["Kenteken"] = "Unknown"
            resultsVar["Zekerheid"] = 100/rowsLeftCount
            resultsVar["Index"] = index
            resultsdf = resultsdf.append(resultsVar, ignore_index=True)
            with open("D:\Traineeship\Data\Output\Results.csv", 'a') as f:
                resultsdf.to_csv(f, header=False)
            index += 1
            break

    if index > df2RowsCount-1:
        print("Done!")
        #resultsdf.to_csv(path_or_buf="D:\Traineeship\Data\Output\Results.csv")
        print(df2Learnt)
        break

